# AtCoder Beginner Contest 180

Created on October 17, 2020  

https://atcoder.jp/contests/abc180

## A（AC）

https://atcoder.jp/contests/abc180/tasks/abc180_a

In [1]:
N, A, B = map(int, input().split())

100 1 2


In [3]:
print(N - A + B)

101


## B（AC）

https://atcoder.jp/contests/abc180/tasks/abc180_b

In [12]:
N = int(input())
X = list(map(int, input().split()))

10
3 -1 -4 1 -5 9 2 -6 5 -3


In [14]:
import math

a1 = sum([abs(x) for x in X])
a2 = math.sqrt(sum([x**2 for x in X]))
a3 = max([abs(x) for x in X])

print(a1)
print(a2)
print(a3)

39
14.38749456993816
9


## C（AC）

https://atcoder.jp/contests/abc180/tasks/abc180_c

In [15]:
N = int(input())

6


In [16]:
def make_divisors(n):
    lower_divisors , upper_divisors = [], []
    i = 1
    while i*i <= n:
        if n % i == 0:
            lower_divisors.append(i)
            if i != n // i:
                upper_divisors.append(n//i)
        i += 1
    return lower_divisors + upper_divisors[::-1]

In [22]:
print(*make_divisors(N), sep='\n')

1
2
3
6


## D（AC）

https://atcoder.jp/contests/abc180/tasks/abc180_d

In [71]:
X, Y, A, B = map(int, input().split())

10 12 2 3


In [72]:
inc_sum = B

exp = 0

while X < Y:
    inc_multi = X*(A-1)
    if inc_multi > inc_sum:
        exp += (Y-X) // B
        if (Y-X) // B == 0:
            break
        else:
            X += B * ((Y-X)//B)
    else:
        X *= A
        exp += 1

if X >= Y:
    print(exp-1)
else:
    print(exp)

0


## E（WA）

In [158]:
N = int(input())
c = []
for _ in range(N):
    c.append(list(map(int, input().split())))

3
0 0 0
1 1 1
-1 -1 -1


In [172]:
from ipywidgets import Textarea
import io

if 'open' in globals():
    del open
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
open = custom_open()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [201]:
import numpy as np

N, *_c = map(int, open(0).read().split())
c = np.array(list(zip(*[map(int, iter(_c))]*3)))

In [202]:
def dist(a,b,c, p,q,r):
    return abs(p-a) + abs(q-b) + max(0, r-c)

In [203]:
distance_matrix = np.array([[dist(*c[i], *c[j]) for j in range(N)]  for i in range(N)])

In [204]:
def calculate_total_distance(order, distance_matrix):
    """Calculate total distance traveled for given visit order"""
    idx_from = np.array(order)
    idx_to = np.array(order[1:] + [order[0]])
    distance_arr = distance_matrix[idx_from, idx_to]

    return np.sum(distance_arr)

In [205]:
# 試しに距離を計算してみる
test_order = list(np.random.permutation(N))
print('訪問順序 = {}'.format(test_order))

total = calculate_total_distance(test_order, distance_matrix)
print('総移動距離 = {}'.format(total))

訪問順序 = [3, 9, 12, 4, 8, 11, 5, 10, 16, 13, 0, 14, 7, 6, 15, 1, 2]
総移動距離 = 13314247


In [224]:
def calculate_2opt_exchange_cost(visit_order, i, j, distance_matrix):
    """Calculate the difference of cost by applying given 2-opt exchange"""
    n_cities = len(visit_order)
    a, b = visit_order[i], visit_order[(i + 1) % n_cities]
    c, d = visit_order[j], visit_order[(j + 1) % n_cities]

    cost_before = distance_matrix[a, b] + distance_matrix[c, d]
    cost_after = distance_matrix[a, c] + distance_matrix[b, d]
    return cost_after - cost_before

In [225]:
def apply_2opt_exchange(visit_order, i, j):
    """Apply 2-opt exhanging on visit order"""

    tmp = visit_order[i + 1: j + 1]
    tmp.reverse()
    visit_order[i + 1: j + 1] = tmp

    return visit_order

In [226]:
def improve_with_2opt(visit_order, distance_matrix):
    """Check all 2-opt neighbors and improve the visit order"""
    n_cities = len(visit_order)
    cost_diff_best = 0.0
    i_best, j_best = None, None

    for i in range(0, n_cities - 2):
        for j in range(i + 2, n_cities):
            if i == 0 and j == n_cities - 1:
                continue

            cost_diff = calculate_2opt_exchange_cost(
                visit_order, i, j, distance_matrix)

            if cost_diff < cost_diff_best:
                cost_diff_best = cost_diff
                i_best, j_best = i, j

    if cost_diff_best < 0.0:
        visit_order_new = apply_2opt_exchange(visit_order, i_best, j_best)
        return visit_order_new
    else:
        return None

In [259]:
def local_search(visit_order, distance_matrix, improve_func):
    """Main procedure of local search"""
    cost_total = calculate_total_distance(visit_order, distance_matrix)
    judge_num = 0
    
    while True:
        improved = improve_func(visit_order, distance_matrix)
        if not improved:
            break

        visit_order = improved

    return visit_order

In [294]:
# 適当に初期解を生成
test_order = [0, *np.array(list(np.random.permutation(N-1)))+1, 0]
# visualize_visit_order(test_order, city_xy)
total_distance = calculate_total_distance(test_order, distance_matrix)
# print('初期解の総移動距離 = {}'.format(total_distance))

# 近傍を計算
improved = local_search(test_order, distance_matrix, improve_with_2opt)
# visualize_visit_order(improved, city_xy)
total_distance = calculate_total_distance(improved, distance_matrix)
# print('近傍探索適用後の総移動距離 = {}'.format(total_distance))
print(total_distance)

7079527
